# Gas Station Simulator Extension: Requesting Multiple Resources

#### _Edward Krueger, edkrueger@gmail.com_

## Purpose

Show how to have a process request multiple resources in simpy. This is not a default default behavior, so we'll require a work around.

## Overview
We want to add a behavior to Alice's simulator where some cars take up multiple pumps (whether because their cars a very large, because they lack awareness or because they are towing a boat.)

## Simplifying Assumptions

To highlight the behavior we're looking at we'll make some simplifying assumptions. We'll simplify the simulation so that there are a fixed two pumps, which we'll model and report metrics on directly. We'll also make the assumption that cars are assigned randomly to a pump when they arrive at the gas station. (Perhaps some cars require diesel and some require gasoline and each pump only has one type of fuel.) Finally, we'll assume that when a car block both pumps, it always pumps the fuel from the first one. (Perhaps the large cars are all diesel trucks and they block both pumps, but always fill up with the diesel.) Finally, we'll assume that the utilization metric counts for all blocked pumps whether they are pumping or not.

# Gas Station Simulator Solution

## Setup

In [1]:
import simpy

import numpy as np

from scipy.stats import expon
from scipy.stats import norm

from contextlib import ExitStack

In [2]:
class GasPump():
    
    """
    Models a gas pump as a simpy.Resource.
    
    Arguments:
    -----------
    name (str): The name or ID of a pump
    env (simpy.Environment): The simpy environment
    """
    
    def __init__(self, name, env):
        
        self.name = name
                
        self.env = env
        
        self.resource = simpy.Resource(self.env, capacity=1)
        
        self.minutes_utilized = 0
        self.gallons_sold = 0

In [3]:
class Car():

    """
    Models a car.
    The car's simulation process in the simulation is the method .run().
    
    Arguments:
    -----------
    id (int): A unique id for the car
    gas_required (float): Amount of gallons a car will fill
    lay_time (float): Extra minutes that the car stays at the pump
    gas_pump (GasStation): The gas pumps the car is assigned to.
    env (simpy.Environment): The simpy environment
    
    """
    
    def __init__(
        self,
        id,
        gas_required,
        lay_time,
        gas_pumps,
        env
    ):
        
        self.id = id
        self.gas_required = gas_required
        self.lay_time = lay_time
        self.gas_pumps = gas_pumps
        self.env = env
        
        self.wait_time = np.inf
        self.finished = False
        
        self.pump_names = " ".join([gas_pump.name for gas_pump in gas_pumps])
        
        self.action = self.env.process(self.run())
        
    def run(self):
        
        queue_time = self.env.now
        
        print(f'Car {self.id} arrives at {round(queue_time, 2)} minutes')
        
                
        with ExitStack() as stack:
            
            reqs = [
                stack.enter_context(res.resource.request()) for res in self.gas_pumps
            ]
            
            yield self.env.all_of(reqs)
            
            pump_start_time = self.env.now
            
            print(f'Car {self.id} begins utilizing pumps {self.pump_names} at {round(pump_start_time, 2)} minutes')

            pump_time = self.gas_required / PUMP_RATE
            utilization = max(pump_time, self.lay_time)
            
            yield self.env.timeout(utilization)
            
            pump_end_time = self.env.now
            
            print(f'Car {self.id} endutilizing pumps {self.pump_names} at {round(pump_end_time, 2)} minutes')
            
            self.finished = True
            
            self.wait_time = pump_start_time - queue_time
            
            # add the utilization to all pumps
            for gas_pump in self.gas_pumps:
                gas_pump.minutes_utilized += utilization
                
            # add the gas to only the first pump
            self.gas_pumps[0].gallons_sold += self.gas_required

In [4]:
def scheduler(env):
    
    """
    Controls the arrival of cars at the gas station.
    
    Arguments:
    -----------
    env (simpy.Environment): The simpy environment
    """
    
    id = 0
    
    while True:
        
        waiting_time = expon.rvs(loc=0, scale=EXPECTED_WAIT)
        
        std_norm = norm.rvs()
        gas_required = std_norm * GAS_REQUIRED_STD + GAS_REQUIRED_MEAN
        gas_required = max([0, gas_required])
        
        lay_time = expon.rvs(loc=0, scale=EXPECTED_LAY_TIME)
        
        yield env.timeout(waiting_time)
        
        gas_pumps = np.random.choice(
            a = [
                [np.random.choice(pumps)],
                pumps
            ],
            p = [
                1- PROP_LARGE_CARS,
                PROP_LARGE_CARS,
            ]
        )
        
        CARS.append(
            Car(
                id=id,
                gas_required=gas_required,
                lay_time=lay_time,
                gas_pumps=gas_pumps,
                env=env
            )
        )
        
        id += 1

## Input parameters and run the simulation

In [5]:
EXPECTED_WAIT = 2 # in minutes

GAS_REQUIRED_MEAN = 15 # in gallons
GAS_REQUIRED_STD = 5 # in gallons

PUMP_RATE = 10 # in gallons per minute

EXPECTED_LAY_TIME = 5 # in minutes

PROP_LARGE_CARS = .15 # in [0,1]

sim_time = 24 * 60 # in minutes

## Wrap up the simulation and metrics

In [6]:
env = simpy.Environment()
CARS = []

pump_1 = GasPump("1", env)
pump_2 = GasPump("2", env)
pumps = [pump_1, pump_2]

env.process(scheduler(env))
env.run(until=sim_time)


Car 0 arrives at 2.61 minutes
Car 0 begins utilizing pumps 1 2 at 2.61 minutes
Car 1 arrives at 2.69 minutes
Car 2 arrives at 3.13 minutes
Car 3 arrives at 5.1 minutes
Car 4 arrives at 5.69 minutes
Car 5 arrives at 6.48 minutes
Car 0 endutilizing pumps 1 2 at 6.65 minutes
Car 1 begins utilizing pumps 1 2 at 6.65 minutes
Car 6 arrives at 8.35 minutes
Car 7 arrives at 9.09 minutes
Car 1 endutilizing pumps 1 2 at 9.68 minutes
Car 2 begins utilizing pumps 1 at 9.68 minutes
Car 8 arrives at 10.79 minutes
Car 2 endutilizing pumps 1 at 10.91 minutes
Car 3 begins utilizing pumps 1 at 10.91 minutes
Car 9 arrives at 16.03 minutes
Car 10 arrives at 17.79 minutes
Car 11 arrives at 20.24 minutes
Car 3 endutilizing pumps 1 at 20.37 minutes
Car 4 begins utilizing pumps 1 at 20.37 minutes
Car 12 arrives at 21.98 minutes
Car 4 endutilizing pumps 1 at 22.72 minutes
Car 5 begins utilizing pumps 1 2 at 22.72 minutes
Car 13 arrives at 23.47 minutes
Car 14 arrives at 23.88 minutes
Car 5 endutilizing pumps 1

Car 440 arrives at 866.14 minutes
Car 237 endutilizing pumps 1 at 867.32 minutes
Car 238 begins utilizing pumps 1 at 867.32 minutes
Car 441 arrives at 867.65 minutes
Car 442 arrives at 868.41 minutes
Car 238 endutilizing pumps 1 at 868.85 minutes
Car 443 arrives at 869.27 minutes
Car 444 arrives at 871.51 minutes
Car 445 arrives at 873.47 minutes
Car 446 arrives at 874.32 minutes
Car 447 arrives at 874.78 minutes
Car 448 arrives at 875.14 minutes
Car 449 arrives at 877.27 minutes
Car 450 arrives at 877.98 minutes
Car 451 arrives at 878.83 minutes
Car 452 arrives at 880.59 minutes
Car 236 endutilizing pumps 2 at 885.81 minutes
Car 239 begins utilizing pumps 1 2 at 885.81 minutes
Car 239 endutilizing pumps 1 2 at 887.36 minutes
Car 240 begins utilizing pumps 2 at 887.36 minutes
Car 241 begins utilizing pumps 1 at 887.36 minutes
Car 453 arrives at 888.76 minutes
Car 454 arrives at 888.83 minutes
Car 455 arrives at 892.56 minutes
Car 240 endutilizing pumps 2 at 893.02 minutes
Car 241 endut

In [7]:
avg_wait_time = np.array([car.wait_time for car in CARS if car.finished]).mean()
total_gallons_sold = sum([pump.gallons_sold for pump in pumps])

for pump in pumps:
    utilization = pump.minutes_utilized / sim_time * len(pumps) 
    print(f'Utilization for pump {pump.name}: {utilization}')
    
for pump in pumps:
    gallons = pump.gallons_sold
    print(f'Gallons sold at pump {pump.name}: {gallons}')
    


print(f'Average wait time (minutes): {avg_wait_time}')
print(f'Total Gallons Sold: {total_gallons_sold}')

Utilization for pump 1: 1.7006615133198242
Utilization for pump 2: 1.5112939686759845
Gallons sold at pump 1: 3227.9208291063155
Gallons sold at pump 2: 2271.835035350113
Average wait time (minutes): 348.1109931884624
Total Gallons Sold: 5499.755864456429
